In [ ]:
%matplotlib inline

In [ ]:
import numpy as np
import scipy as sp

In [ ]:
from matplotlib import pyplot as plt
import skimage
from skimage import (morphology, segmentation, exposure, feature, filters,
                     measure, transform, util, io, color)

In [ ]:
from toolz.curried import *

In [ ]:
# this will raise a warning about matplotlib backend

from spotz import imgz, gridder, segmenter, spotzplot

# User settings

In [ ]:
# change name of file
PLATE_IMG = "XXXX"

# change number of rows and cols as appropriate
nrows = 12
ncols = 8

# Original image

In [ ]:
img = imgz.read_image(PLATE_IMG)

In [ ]:
fig = plt.figure(figsize = (10,6))
imgz.imshowg(img)
pass

# Invert image

In [ ]:
iimg = imgz.invert(img)

In [ ]:
fig = plt.figure(figsize = (10,6))
imgz.imshowg(iimg)
pass

# Gridding

## Thresholding and binary opening

In [ ]:
bimg, elemsize = gridder.threshold_and_open(iimg,
                                           threshold_func = imgz.threshold_otsu)

In [ ]:
fig = plt.figure(figsize = (10,6))
imgz.imshowg(bimg)
pass

## Estimate grid coordinates and draw grid

In [ ]:
grid_data = gridder.find_grid(bimg, nrows, ncols, min_n=1)

In [ ]:
fig = plt.figure(figsize = (10,6))
imgz.imshowg(bimg)
ax = plt.gca()
spotzplot.draw_bboxes(grid_data["bboxes"], ax)
pass

# Segmentation

## Mask areas outside grid

In [ ]:
mbimg = imgz.mask_outside_bbox(grid_data["total_bbox"], bimg)

In [ ]:
fig = plt.figure(figsize = (10,6))
imgz.imshowg(mbimg)
pass

## Segmentation by watershed

In [ ]:
edges = filters.scharr(iimg)
seeds = np.zeros_like(iimg, dtype = int)
wshed = np.zeros_like(iimg, dtype = np.uint32)

# watershed each grid unit independently
for i, ctr in enumerate(grid_data["centers"]):
    ctr = tuple(ctr)
    bbox = grid_data["bboxes"][i]
    minr, minc, maxr, maxc = bbox
    if mbimg[ctr]:
        seeds[ctr] = i + 1
        unit_edges = edges[minr:maxr, minc:maxc]
        unit_seed = seeds[minr:maxr, minc:maxc]
        wshed[minr:maxr, minc:maxc] = segmenter.segment_grid_unit(unit_edges, unit_seed)


In [ ]:
fig = plt.figure(figsize = (10,6))
ax = plt.gca()
ax.imshow(color.label2rgb(wshed, img, bg_label = 0))
pass

In [ ]:
fig, ax = spotzplot.draw_image_and_labels(img, wshed,
                                          mask_cmap = "Reds", alpha = 0.35,
                                         fontsize = 10, textcolor = "orange")
fig.set_size_inches(10,6)